In [1]:
n = 3
n_vars = n*(n+1)/2
var = ['s%i%i'%(i,j) for j in range(1,n+1) for i in range(1,j+1)]
var+= ['g%i%i'%(i,j) for i in range(1,n_vars+1) for j in range(1,n_vars+1)]
R = PolynomialRing(QQ, var)
R.inject_variables()

Defining s11, s12, s22, s13, s23, s33, g11, g12, g13, g14, g15, g16, g21, g22, g23, g24, g25, g26, g31, g32, g33, g34, g35, g36, g41, g42, g43, g44, g45, g46, g51, g52, g53, g54, g55, g56, g61, g62, g63, g64, g65, g66


In [4]:
gens = [s12*s13-s11*s23, s12^2-s11*s22-s13^2+s11*s33]
show(gens[0])
show(gens[1])

s12*s13 - s11*s23

s12^2 - s11*s22 - s13^2 + s11*s33

In [3]:
degs = list(WeightedIntegerVectors(2,[1 for i in range(n_vars)]))
n_2 = binomial(n_vars+1,2)
monomials_2 = []
for d in degs:
    mon = 1
    for i in range(len(R.gens()[:n_vars])):
        mon = mon*R.gens()[i]^d[i]
    monomials_2.append(mon)
gen_index = {}
count = 0
for i in R.gens():
    gen_index[i] = count
    count+= 1
X = matrix(R,n_vars,1,R.gens()[:n_vars])

In [4]:
g = matrix(R,n_vars,n_vars,R.gens()[n_vars:])
g

[g11 g12 g13 g14 g15 g16]
[g21 g22 g23 g24 g25 g26]
[g31 g32 g33 g34 g35 g36]
[g41 g42 g43 g44 g45 g46]
[g51 g52 g53 g54 g55 g56]
[g61 g62 g63 g64 g65 g66]

In [5]:
def lie_derivative_mono(monomial, g):
    s = 0
    for x in monomial.factor():
        l = (g[gen_index[x[0]],:]*X)[0,0]
        s+= l*R(monomial/x[0])*x[1]
    return s
def lie_derivative_poly(poly,g):
    der = 0
    for mon, coef in list(zip(poly.monomials(),poly.coefficients())):
        der+= lie_derivative_mono(mon,g)*coef
    return der
def poly_to_vec(polynomial):
    vec = [0 for j in range(n_2)]
    for i in range(n_2):
        vec[i] = polynomial.coefficient(monomials_2[i])
    return vec

In [6]:
gens_vecs = [poly_to_vec(gen) for gen in gens]
der_gens_vecs = [poly_to_vec(lie_derivative_poly(gen,g)) for gen in gens]
matrices = [matrix(gens_vecs+[i]) for i in der_gens_vecs]

In [7]:
eqns = set()
for M in matrices:
    for eqn in M.minors(len(gens)+1):
        if eqn!= 0:
            eqns.add(eqn)
eqns = list(eqns)

In [8]:
ker = []
for eqn in eqns:
    l = [eqn.coefficient(gij) for gij in R.gens()[n_vars:]]
    ker.append(l)
K = matrix(ker)
rows = K.pivot_rows()
K = K[rows,:]
K.dimensions()

(25, 36)

In [9]:
K_ker = K.right_kernel_matrix()
K_ker.dimensions()

(11, 36)

In [10]:
L = [matrix(QQ,n_vars,n_vars,r) for r in K_ker.rows()]
show(L)

[
[ 1  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 1  0  0  0  0  0]  [ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  1  0  0]
[ 0  0  0  0 -1  0]  [ 0  0  0  1  0  0]  [ 0  0  0  0  2  0]
[ 0  0  1  0  0 -1], [ 0 -2  0  0  0  0], [ 0  0 -2  0  0  2],

[ 0  0  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  1  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0  0  0]  [1 0 0 0 0 0]  [0 1 0 0 0 0]  [0 0 1 0 0 0]
[ 0 -1  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0 -1  0  0  1]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0 -4  0], [1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 1 0 0]  [0 0 0 0 1 0]  [0 0 0 0 0 1]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 1 0 0 0 0]
[0 0 0 1 0 0], [0 0 0 0 1 0], [0 0 0 0 0 1], [0 0 0 2 0 0]
]

In [7]:
load('Symmalg.sage')
#LieI = gap.LieAlgebra(gap.Rationals,L)
LieI = symmalg(gens, n=6)

Defining s11, s12, s22, s13, s23, s33
Defining s11, s12, s22, s13, s23, s33, g11, g12, g13, g14, g15, g16, g21, g22, g23, g24, g25, g26, g31, g32, g33, g34, g35, g36, g41, g42, g43, g44, g45, g46, g51, g52, g53, g54, g55, g56, g61, g62, g63, g64, g65, g66

A basis of the Lie algebra consists of the following matrices:



[
[ 1  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 1  0  0  0  0  0]  [ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  1  0  0]
[ 0  0  0  0 -1  0]  [ 0  0  0  1  0  0]  [ 0  0  0  0  2  0]
[ 0  0  1  0  0 -1], [ 0 -2  0  0  0  0], [ 0  0 -2  0  0  2],

[ 0  0  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  1  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0  0  0]  [1 0 0 0 0 0]  [0 1 0 0 0 0]  [0 0 1 0 0 0]
[ 0 -1  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0 -1  0  0  1]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0 -4  0], [1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 1 0 0]  [0 0 0 0 1 0]  [0 0 0 0 0 1]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 1 0 0 0 0]
[0 0 0 1 0 0], [0 0 0 0 1 0], [0 0 0 0 0 1], [0 0 0 2 0 0]
]

In [8]:
LieI.Dimension()

11

In [10]:
LieI.IsLieSolvable()

true

In [22]:
L = []
for m in LieI.Basis():
    L.append(matrix(QQ,m))
show(L)

[
[ 1  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 1  0  0  0  0  0]  [ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]  [ 0  0  0  0  0  0]  [ 0  0  0  1  0  0]
[ 0  0  0  0 -1  0]  [ 0  0  0  1  0  0]  [ 0  0  0  0  2  0]
[ 0  0  1  0  0 -1], [ 0 -2  0  0  0  0], [ 0  0 -2  0  0  2],

[ 0  0  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  1  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0  0  0]  [1 0 0 0 0 0]  [0 1 0 0 0 0]  [0 0 1 0 0 0]
[ 0 -1  0  0  0  0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0 -1  0  0  1]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[ 0  0  0  0 -4  0], [1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0],

[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]
[0 0 0 1 0 0]  [0 0 0 0 1 0]  [0 0 0 0 0 1]  [0 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [1 0 0 0 0 0]
[0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 0 0 0 0 0]  [0 1 0 0 0 0]
[0 0 0 1 0 0], [0 0 0 0 1 0], [0 0 0 0 0 1], [0 0 0 2 0 0]
]

In [27]:
K = QQ[sqrt(-1)]
V = VectorSpace(K,6)
espaces = []
for l in L:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)


In [34]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
Spaces

[Vector space of degree 6 and dimension 1 over Number Field in I with defining polynomial x^2 + 1 with I = 1*I
 Basis matrix:
 [0 0 1 0 0 1]]

In [31]:
I1 = matrix.identity(6)
I1 = I1.change_ring(K)
I1[:,2] = v
t = I1[:,0]
I1[:,0] = I1[:,2]
I1[:,2] = t
show(I1)
L1 = []
for l in L:
    k = I1.inverse()*l*I1
    L1.append(k[1:,1:])
    show(k)

[0 0 1 0 0 0]
[0 1 0 0 0 0]
[1 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[1 0 0 0 0 1]

[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0 -1  0]
[ 0  0  0  0  0 -1]

[ 0  0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0 -2  0  0  0  0]

[0 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 2 0]
[0 0 0 0 0 2]

[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0  0  0  0  1]
[ 0  0  0  0 -4  0]

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 0 1 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[1 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 1 0 0 0]
[0 1 0 0 0 0]
[0 0 0 2 0 0]

In [35]:
V = VectorSpace(K,5)
espaces = []
for l in L1:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [36]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp

In [37]:
v = Spaces[1].basis()
vm = vector(K,v[0])
I2 = matrix.identity(5)
I2 = I2.change_ring(K)
I2[:,3] = v[0]
t = I2[:,3]
I2[:,3] = I2[:,0]
I2[:,0] = t
show(I2)
L2 = []
for l in L1:
    k = I2.inverse()*l*I2
    L2.append(k[1:,1:])
    show(k)


[   0    0    0    1    0]
[   0    1    0    0    0]
[   0    0    1    0    0]
[   1    0    0    0    0]
[-2*I    0    0    0    1]

[-1  0  0  0  0]
[ 0  1  0  0  0]
[ 0  0  0  0  0]
[ 0  0  0  0  0]
[ 0  0  0  0 -1]

[  0   0   1   0   0]
[  0   0   0   0   0]
[  0   0   0   0   0]
[  0   1   0   0   0]
[  0   0 2*I  -2   0]

[2 0 0 0 0]
[0 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 2]

[-2*I    0    0    0    1]
[   0    0    0    0    0]
[   0    0    0   -1    0]
[   0    0    1    0    0]
[   0    0    0    0  2*I]

...

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[  0   0   0   1   0]
[  0   0   0   0   0]
[  0   1   0   0   0]
[  0   0   0   0   0]
[  0   0   2 2*I   0]

In [38]:
V = VectorSpace(K,4)
espaces = []
for l in L2:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [39]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
v

(0, 0, 0, 1)

In [41]:
I3 = matrix.identity(4)
I3 = I3.change_ring(K)
I3[:,3] = I3[:,0]
I3[:,0] = v
L3 = []
show(I3)
for l in L2:
    k = I3.inverse()*l*I3
    L3.append(k[1:,1:])
    show(k)

[0 0 0 1]
[0 1 0 0]
[0 0 1 0]
[1 0 0 0]

[-1  0  0  0]
[ 0  0  0  0]
[ 0  0  0  0]
[ 0  0  0  1]

[  0 2*I  -2   0]
[  0   0   0   0]
[  0   0   0   1]
[  0   0   0   0]

[2 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 0]

[2*I   0   0   0]
[  0   0  -1   0]
[  0   1   0   0]
[  0   0   0   0]

...

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

[  0   2 2*I   0]
[  0   0   0   1]
[  0   0   0   0]
[  0   0   0   0]

In [42]:
V = VectorSpace(K,3)
espaces = []
for l in L3:
    espace = []
    for S in l.right_eigenspaces():
        espace.append(V.subspace(S[1]))
    espaces.append(espace)

In [43]:
Spaces = espaces[0]
for e in espaces:
    temp = []
    for v in e:
        for s in Spaces:
            t = v.intersection(s)
            if dim(t)!=0 and t not in temp:
                temp.append(t)
    Spaces = temp
v = Spaces[0].basis()[0]
v

(1, -I, 0)

In [44]:
I4 = matrix.identity(3)
I4 = I4.change_ring(K)
I4[:,0] = v
show(I4)
L4 = []
for l in L3:
    k = I4.inverse()*l*I4
    L4.append(k[1:,1:])
    show(k)

[ 1  0  0]
[-I  1  0]
[ 0  0  1]

[0 0 0]
[0 0 0]
[0 0 1]

[0 0 0]
[0 0 1]
[0 0 0]

[1 0 0]
[0 1 0]
[0 0 0]

[ I -1  0]
[ 0 -I  0]
[ 0  0  0]

...

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 0]
[0 0 0]
[0 0 0]

[0 0 1]
[0 0 I]
[0 0 0]

In [48]:
I2_1 = matrix.identity(K,6)
I2_1[1:,1:] = I2
I3_1 = matrix.identity(K,6)
I3_1[2:,2:] = I3
I4_1 = matrix.identity(K,6)
I4_1[3:,3:] = I4
I = I1*I2_1*I3_1*I4_1
L_upper = []
for l in L:
    k = I.inverse()*l*I
    L_upper.append(k)
    show(k)

[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1]

[  0   0   0   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0 4*I  -2   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   0]

[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 0 2 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0]

[   0    0    0    0    0    0]
[   0 -2*I    1    0    0    0]
[   0    0  2*I    0    0    0]
[   0    0    0    I   -1    0]
[   0    0    0    0   -I    0]
[   0    0    0    0    0    0]

[0 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

[   1 -2*I    1    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]

[  0   0   0   0   0   0]
[  0   0   0  -I   1   0]
[  0   0   0   4 2*I   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   I]
[  0   0   0   0   0   0]

In [49]:
X = matrix(R,n_vars,1,R.gens()[:n_vars])
#vector(X)*g
sub = {}
for i in range(6):
    sub[X[i][0].change_ring(K)] = (vector(X)*I.inverse())[i]
sub

{s11: s33,
 s12: s23,
 s22: s11 - s22,
 s13: s13 + I*s23,
 s23: s12 + (2*I)*s22,
 s33: s22}

In [50]:
p1 = (gens[0].change_ring(K))
show(p1.subs(sub))
p2 = (gens[1].change_ring(K))
show(p2.subs(sub))

s13*s23 + I*s23^2 - s12*s33 + (-2*I)*s22*s33

-s13^2 + (-2*I)*s13*s23 + 2*s23^2 - s11*s33 + 2*s22*s33

In [36]:
L_upper[3].minpoly().roots()

[(2*I, 1), (I, 1), (0, 1), (-I, 1), (-2*I, 1)]

In [37]:
for l in L_upper:
    print(l.is_diagonalizable())

True
False
True
True
False
False
True
False
False
True
False


In [38]:
L_upper[10].is_diagonalizable()

False

In [51]:
for l in L_upper:
    show(l, l.is_diagonalizable())

[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1] True

[  0   0   0   0   0   0]
[  0   0   0   1   0   0]
[  0   0   0 4*I  -2   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   0] False

[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 0 2 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0] True

[   0    0    0    0    0    0]
[   0 -2*I    1    0    0    0]
[   0    0  2*I    0    0    0]
[   0    0    0    I   -1    0]
[   0    0    0    0   -I    0]
[   0    0    0    0    0    0] True

[0 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

...

[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] True

[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] False

[   1 -2*I    1    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0] True

[  0   0   0   0   0   0]
[  0   0   0  -I   1   0]
[  0   0   0   4 2*I   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   I]
[  0   0   0   0   0   0] False

In [54]:
show(L_upper[-1], L_upper[3])

[  0   0   0   0   0   0]
[  0   0   0  -I   1   0]
[  0   0   0   4 2*I   0]
[  0   0   0   0   0   1]
[  0   0   0   0   0   I]
[  0   0   0   0   0   0] [   0    0    0    0    0    0]
[   0 -2*I    1    0    0    0]
[   0    0  2*I    0    0    0]
[   0    0    0    I   -1    0]
[   0    0    0    0   -I    0]
[   0    0    0    0    0    0]

In [61]:
P = L_upper[3].diagonalization()[1]
P.inverse()*L_upper[3]*P

[ 2*I    0    0    0    0    0]
[   0    I    0    0    0    0]
[   0    0   -I    0    0    0]
[   0    0    0 -2*I    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]

In [62]:
for l in L_upper:
    show(P.inverse()*l*P, l.is_diagonalizable())

[-1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0 -1  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1] True

[     0      1      0      0      0      0]
[     0      0      0      0      0  1/2*I]
[     0      0      0      0      0 -1/2*I]
[     0      0      1      0      0      0]
[     0      0      0      0      0      0]
[     0      0      0      0      0      0] False

[2 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 2 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] True

[ 2*I    0    0    0    0    0]
[   0    I    0    0    0    0]
[   0    0   -I    0    0    0]
[   0    0    0 -2*I    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1]
[0 0 0 0 0 0] False

...

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 1 1 0 0 0]
[0 0 0 0 0 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 1 0 0]
[0 0 0 0 0 0] False

[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[ 2*I    0    0 -2*I    1    0]
[   0    0    0    0    0    0] True

[  0  -I   0   0   0   0]
[  0   0   0   0   0 1/2]
[  0   0   0   0   0 1/2]
[  0   0   I   0   0   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   0] False

In [64]:
G = I*P
sub = {}
for i in range(6):
    sub[X[i][0].change_ring(K)] = (G*vector(X))[i]
show(sub)
p1 = (gens[0].change_ring(K))
show(p1.subs(sub))
p2 = (gens[1].change_ring(K))
show(p2.subs(sub))

{s11: s33,
 s12: (-I)*s12 + I*s22,
 s22: s23,
 s13: s12 + s22,
 s23: s11 + s13,
 s33: (2*I)*s11 + (-2*I)*s13 + s23}

(-I)*s12^2 + I*s22^2 - s11*s33 - s13*s33

-2*s12^2 - 2*s22^2 + (2*I)*s11*s33 + (-2*I)*s13*s33

In [65]:
show(G)

[   0    0    0    0    0    1]
[   0   -I    I    0    0    0]
[   0    0    0    0    1    0]
[   0    1    1    0    0    0]
[   1    0    0    1    0    0]
[ 2*I    0    0 -2*I    1    0]

In [72]:
L = [l.change_ring(K) for l in L]
for l in L:
    show(l, l.is_diagonalizable())

[ 1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0 -1  0]
[ 0  0  1  0  0 -1] True

[ 0  0  0  0  0  0]
[ 1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0 -2  0  0  0  0] False

[ 0  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0  0  0  0  2  0]
[ 0  0 -2  0  0  2] True

[ 0  0  0  0  0  0]
[ 0  0  0  1  0  0]
[ 0  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  1]
[ 0  0  0  0 -4  0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0] False

...

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 1 0 0 0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 1 0 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 2 0 0] False

In [73]:
P1 = L[0].diagonalization()[1]
for l in L:
    l1 = P1.inverse()*l*P1
    show(l1, l1.is_diagonalizable())

[ 1  0  0  0  0  0]
[ 0 -1  0  0  0  0]
[ 0  0 -1  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0] True

[ 0  0  0  0  0  0]
[ 0  0  0  0  0  1]
[ 0  0  0 -2  0  0]
[ 1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0] False

[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 0 2 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1] True

[ 0  0  0  0  0  0]
[ 0  0  1  0  0  0]
[ 0 -4  0  0  0  0]
[ 0  0  0  0  0  1]
[ 0  0  0  0  0  0]
[ 0  0  0 -1  0  0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 0 0 0 0 0] False

...

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1]
[0 0 0 0 0 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 0 0 0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 1 0 1 0]
[0 0 0 0 0 0] True

[0 0 0 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 0 2]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0] False

In [77]:
L_1 = [P1.inverse()*l*P1 for l in L]
P2 = L_1[3].diagonalization()[1]


[   0    0    0    0    1    0]
[   1    0    0    1    0    0]
[ 2*I    0    0 -2*I    0    0]
[   0    1    1    0    0    0]
[   0    0    0    0    0    1]
[   0    I   -I    0    0    0]

In [80]:
for l in L_1:
    l1 = P2.inverse()*l*P2
    show(l1, l1.is_diagonalizable())

[-1  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0 -1  0  0]
[ 0  0  0  0  1  0]
[ 0  0  0  0  0  0] True

[  0   I   0   0   0   0]
[  0   0   0   0 1/2   0]
[  0   0   0   0 1/2   0]
[  0   0  -I   0   0   0]
[  0   0   0   0   0   0]
[  0   0   0   0   0   0] False

[2 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 2 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0] True

[ 2*I    0    0    0    0    0]
[   0    I    0    0    0    0]
[   0    0   -I    0    0    0]
[   0    0    0 -2*I    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0] True

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 1 0] False

...

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 1] True

[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  0  0  0  0  0]
[ 0  I -I  0  0  0] False

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 1 0 0] False

[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[   0    0    0    0    0    0]
[ 2*I    0    0 -2*I    0    1] True

[     0      1      0      0      0      0]
[     0      0      0      0 -1/2*I      0]
[     0      0      0      0  1/2*I      0]
[     0      0      1      0      0      0]
[     0      0      0      0      0      0]
[     0      0      0      0      0      0] False

In [82]:
P2*P1
G = P2*P1
sub = {}
for i in range(6):
    sub[X[i][0].change_ring(K)] = (G*vector(X))[i]
show(sub)
p1 = (gens[0].change_ring(K))
show(p1.subs(sub))
p2 = (gens[1].change_ring(K))
show(p2.subs(sub))

{s11: s12,
 s12: s11 + s33,
 s22: (2*I)*s11 + (-2*I)*s33,
 s13: s13 + s23,
 s23: s22 + s23,
 s33: I*s13 + (-I)*s23}

-s12*s22 + s11*s13 + s11*s23 - s12*s23 + s13*s33 + s23*s33

s11^2 + (-2*I)*s11*s12 + I*s12*s13 - s13^2 + (-I)*s12*s23 - 2*s13*s23 - s23^2 + 2*s11*s33 + (2*I)*s12*s33 + s33^2